In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import cv2
import glob
import os
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree
import pickle
from tqdm import tqdm
import das.imu as DI
from das.api import platform, interpolators
from leddar_utils import linalg
import extrinsic_ouster64_camera as E
import averagingSO3 as so

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
#Define the Checkerboard
checkerboard = E.Checkerboard(pattern=(4,3), square_length=0.145)
checkerboard.points

array([[0.   , 0.   , 0.   ],
       [0.145, 0.   , 0.   ],
       [0.29 , 0.   , 0.   ],
       [0.435, 0.   , 0.   ],
       [0.   , 0.145, 0.   ],
       [0.145, 0.145, 0.   ],
       [0.29 , 0.145, 0.   ],
       [0.435, 0.145, 0.   ],
       [0.   , 0.29 , 0.   ],
       [0.145, 0.29 , 0.   ],
       [0.29 , 0.29 , 0.   ],
       [0.435, 0.29 , 0.   ]], dtype=float32)

In [4]:
#Load datasets, platforms, and frames range:
__datasets=[
    '/nas/pixset/exportedDataset/20200325_145233_rec_dataset_calib_jeanne01_exported/',
    '/nas/pixset/exportedDataset/20200325_151131_rec_dataset_calib_paul01_exported/',
    '/nas/pixset/exportedDataset/20200407_112921_rec_dataset_calib_jeanne02_gd_carreaux_exported/',
    '/nas/pixset/exportedDataset/20200407_124505_rec_dataset_calib_paul02_gd_carreaux_exported/',
    '/nas/pixset/exportedDataset/20200502_172647_rec_dataset_pierre01_grands_carreaux_exported/',
    '/nas/pixset/exportedDataset/20200502_172126_rec_dataset_jeanne03_grands_carreaux_exported/'
] 

pfs = [platform.Platform(omega) for omega in __datasets]

source = 'ouster64_bfc_xyzit'
destination = 'flir_bfc_img'

  0%|          | 0/6 [00:00<?, ?it/s]/home/olivier/workspace/das/das/api/sensors.py:687: UserWarning: Unable to read completely data from cfg file
  warnings.warn("Unable to read completely data from cfg file")
 17%|█▋        | 1/6 [00:00<00:00,  6.21it/s]/home/olivier/workspace/das/das/api/filesource.py:261: UserWarning: Timestamps are not strictly increasing for datasource file /nas/pixset/exportedDataset/20200325_145233_rec_dataset_calib_jeanne01_exported/flir_bfr_img.zip
  warnings.warn('Timestamps are not strictly increasing for datasource file {}'.format(self.path))
 33%|███▎      | 2/6 [00:00<00:00,  6.68it/s]

Pixell Log: The cache is set to use calibration angles.


 33%|███▎      | 2/6 [00:00<00:00,  6.23it/s]

Pixell Log: The cache is set to use calibration angles.


 50%|█████     | 3/6 [00:00<00:00,  6.01it/s]/home/olivier/workspace/das/das/api/filesource.py:261: UserWarning: Timestamps are not strictly increasing for datasource file /nas/pixset/exportedDataset/20200325_151131_rec_dataset_calib_paul01_exported/flir_bfl_img.zip
  warnings.warn('Timestamps are not strictly increasing for datasource file {}'.format(self.path))
 33%|███▎      | 2/6 [00:00<00:00,  5.86it/s]

...

 50%|█████     | 3/6 [00:00<00:00,  5.65it/s]/home/olivier/workspace/das/das/api/filesource.py:261: UserWarning: Timestamps are not strictly increasing for datasource file /nas/pixset/exportedDataset/20200407_124505_rec_dataset_calib_paul02_gd_carreaux_exported/flir_bfl_img.zip
  warnings.warn('Timestamps are not strictly increasing for datasource file {}'.format(self.path))
 33%|███▎      | 2/6 [00:00<00:00,  6.82it/s]

Pixell Log: The cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


 33%|███▎      | 2/6 [00:00<00:00,  6.04it/s]

Pixell Log: The cache is set to use calibration angles.
Pixell Log: time base delays are converted to lcas referential.


100%|██████████| 6/6 [00:00<00:00,  8.36it/s]


In [5]:
#frames of interest from flir_bfc in same order as platform:
__frames_bfc = [
    np.concatenate((np.arange(2,11),
                        np.arange(37,131),
                        np.arange(175,187),
                        np.arange(241,426))),
    np.concatenate((np.arange(0,40),
                        np.arange(75,80),
                        np.arange(111,166),
                        np.arange(250,310),
                        np.arange(353,439))),
    np.arange(1,192),
    np.concatenate((np.arange(0,264),
                        np.arange(397,404))),
    np.concatenate((np.arange(15,24),
                           np.arange(33,40),
                           np.arange(53,69),
                           np.arange(75,92),
                           np.arange(109,120),
                           np.arange(129,140))),
    np.concatenate((np.arange(3,9),
                           np.arange(88,95),
                           np.arange(124,133),
                           np.arange(150,165),
                           np.arange(193,250),
                           np.arange(254,258),
                           np.arange(292,314)))
]

In [6]:
#Parameters of initial guess:
#The homogenous transformation from flir_camera to camera can be obtained from dasview, or by
# solving a single frame with SVD.

#Camera must have intrinsic calib first:
mtx, dist = E.load_intrinsic_camera(__datasets[5]+'intrinsics/flir_bfc_calib-results.pkl')

with open(__datasets[0]+'extrinsics/ouster64_tfc-flir_bfc.pkl', 'rb') as g:
    tf_from_ouster_to_flir_bfc = pickle.load(g)

tf_from_flir_bfc_to_ouster = linalg.tf_inv(tf_from_ouster_to_flir_bfc)
r, t = tf_from_flir_bfc_to_ouster[:3,:3], tf_from_flir_bfc_to_ouster[:3,3:]

In [7]:
mtx, dist, r, t

(array([[820.24753975,   0.        , 734.30704743],
        [  0.        , 820.78873052, 552.62262377],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-0.22649187,  0.10065042, -0.00026163, -0.00022689, -0.02206318]]),
 array([[ 6.06826773e-03, -2.02354674e-05,  9.99981588e-01],
        [-9.99947570e-01, -8.24850946e-03,  6.06789438e-03],
        [ 8.24823480e-03, -9.99965980e-01, -7.02885703e-05]]),
 array([[ 0.13941018],
        [-0.00108739],
        [-0.10272664]]))

In [8]:
DI.print_transf_result(tf_from_flir_bfc_to_ouster)

translations :[ 0.13941018 -0.00108739 -0.10272664], norm:0.17317373549655454
Ax Angle vector:[-0.58127064  0.57300802 -0.57774238], angle (deg):120.07447490928172
Euler Angle (deg):[-90.00402738  -0.4725944  -89.65229991]


In [9]:
prior_bfc = E.ProtoCameraToLidar(checkerboard, mtx, dist, r,t,'c', epsilons=(1.5,1.25,1.25))

In [21]:
#Init params:
corners_cam_uv = []
corners_ous = []
corners_cam = []


In [22]:
corners_mask = []
corners_mask_ = [
    0,1,1,0,
    0,1,1,0,
    0,1,1,0
]
#corners_mask_ = [
#    0,0,0,0,
#    0,1,1,0,
#    0,0,0,0
#]

In [23]:
#iterate over all frames:
frames =  __frames_bfc[5]#[15:17]
pf = pfs[5]
SHOW_ = False
for mu in tqdm(frames):
    sample_flir = pf[destination][mu]
    sample_lidar = pf[source].get_at_timestamp(sample_flir.timestamp,
                                             interpolator=interpolators.nearest_interpolator)
    
    img = np.copy(sample_flir.raw)
    X = sample_lidar.point_cloud()
    A = sample_lidar.amplitudes
    
    ret_cam, corners_cam_uv_ = E.get_camera_corners(img, 
                                                    checkerboard, 
                                                    SHOW_)
   
    if ret_cam:
        try:
            corners_cam_uv_ = corners_cam_uv_[::-1,:,:]
            ret_ous, corners_ous_, corners_cam_ = E.get_lidar_corners(X, A, prior_bfc, 
                                                            corners_cam_uv_, 
                                                            SHOW_, 
                                                            tolerance_eps=0.020, 
                                                            tolerance_nedges=7)
            if ret_ous:
                corners_cam_uv.append(corners_cam_uv_)
                corners_cam.append(corners_cam_)
                corners_ous.append(corners_ous_)
                corners_mask.append(corners_mask_)
        except:
            pass


  1%|          | 1/120 [00:00<00:24,  4.88it/s]

Log: 3D points for ouster64_bfc are corrected with T 25.94: global offset is -0.15



  2%|▏         | 2/120 [00:00<00:23,  4.93it/s]

Log: 3D points for ouster64_bfc are corrected with T 25.94: global offset is -0.15
Log: 3D points for ouster64_bfc are corrected with T 25.94: global offset is -0.15



  3%|▎         | 4/120 [00:02<01:20,  1.45it/s]

...


 98%|█████████▊| 118/120 [02:38<00:03,  1.75s/it]

Log: 3D points for ouster64_bfc are corrected with T 26.19: global offset is -0.15
Log: 3D points for ouster64_bfc are corrected with T 26.19: global offset is -0.15



 99%|█████████▉| 119/120 [02:39<00:01,  1.49s/it]

Log: 3D points for ouster64_bfc are corrected with T 26.19: global offset is -0.15



100%|██████████| 120/120 [02:40<00:00,  1.34s/it]


In [ ]:
#Once all frames have been selected, do the following:

In [24]:
stack_corners_cam_uv = np.vstack(corners_cam_uv).squeeze(1)
stack_corners_cam = np.vstack(corners_cam)
stack_corners_ous = np.vstack(corners_ous)
stack_corners_mask = np.hstack(corners_mask).astype(bool)

In [25]:
stack_corners_cam_uv.shape, stack_corners_cam.shape, stack_corners_ous.shape

((1152, 2), (1152, 3), (1152, 3))

In [26]:
stack_corners_cam_uv = stack_corners_cam_uv[stack_corners_mask]
stack_corners_ous = stack_corners_ous[stack_corners_mask]
stack_corners_cam = stack_corners_cam[stack_corners_mask]

In [27]:
#display the space of points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(stack_corners_cam[:,0], stack_corners_cam[:,1], stack_corners_cam[:,2], s=50, color='orange', marker='d', label='camera points')
ax.scatter(stack_corners_ous[:,0], stack_corners_ous[:,1], stack_corners_ous[:,2], s=50,color='green' , marker='o', label='ouster points')
plt.legend()
plt.show()

In [28]:
#using svd to obtain affine transformation from ouster to camera:
Rot_,Trans_=E.euclidean_transform_3D(stack_corners_ous, stack_corners_cam)

In [29]:
Rot_, Trans_

(array([[-7.52183604e-04, -9.99985841e-01,  5.26798341e-03],
        [ 1.58678639e-03, -5.26917181e-03, -9.99984859e-01],
        [ 9.99998458e-01, -7.43813051e-04,  1.59072731e-03]]),
 array([ 0.0168517 , -0.10350207, -0.15225468]))

In [30]:
#replace svd by pnp ransac
f, r_, t_, a = cv2.solvePnPRansac(stack_corners_ous, stack_corners_cam_uv, mtx, dist, 
                                          iterationsCount=1500, reprojectionError=1.5)
r_, _ = cv2.Rodrigues(r_)
f,len(a)

(True, 160)

In [31]:
r_, t_

(array([[ 1.43941155e-03, -9.99981705e-01,  5.87523992e-03],
        [ 9.20798323e-06, -5.87523275e-03, -9.99982741e-01],
        [ 9.99998964e-01,  1.43944080e-03,  7.50936889e-07]]),
 array([[ 0.00800188],
        [-0.09493692],
        [-0.12493336]]))

In [32]:
# Transformation Matrix.
M = np.eye(4)
M[:3,:3] = r_
M[:3,3:]= t_

In [33]:
#Save result:
pickle.dump(M, open("ouster64_bfc-flir_bfc.pkl", "wb"))

In [34]:
DI.print_transf_result(M)

translations :[ 0.00800188 -0.09493692 -0.12493336], norm:0.15711585627337732
Ax Angle vector:[ 0.57902991 -0.57480989  0.57820234], angle (deg):120.14681971058945
Euler Angle (deg):[ 89.97010957 -89.91752608   0.36651879]


In [36]:
#Evaluation of error:
ous_to_cam = (Rot_.dot(stack_corners_ous.T) + Trans_[:,None]).T
err_ = E.lj_error(ous_to_cam,stack_corners_cam)
plt.figure()
plt.hist(err_, bins=100)
plt.show()